In [1]:
%rm  test/*
from polychrom.simulation import Simulation
import polychrom.starting_conformations
import polychrom.forces as forces
import polychrom.forcekits as forcekits
import polychrom.polymerutils

from polychrom.hdf5_format import HDF5Reporter

data = polychrom.starting_conformations.grow_cubic(100,30)

"""
Here we created a hdf5Reporter attached to a foler test, and we are saving 5 blocks per file 
(you should probalby use 50 here or 100. 5 is just for a showcase)
"""
reporter = HDF5Reporter(folder="test", overwrite=True)


"""
Passing a reporter to the simulation object - many reporters are possible, and more will be added in a future
"""
sim = Simulation(N=100, error_tol=0.001, collision_rate=0.1, integrator ="variableLangevin", platform="CPU", 
                reporters=[reporter])
sim.set_data(data)
sim.add_force(
    forcekits.polymer_chains(
        sim,
        chains=[(0, 100,True)],
            # By default the library assumes you have one polymer chain
            # If you want to make it a ring, or more than one chain, use self.setChains
            # self.setChains([(0,50,True),(50,None,False)]) will set a 50-monomer ring and a chain from monomer 50 to the end

        bond_force_func=forces.grosberg_polymer_bonds,
        bond_force_kwargs={},

        angle_force_func=forces.grosberg_angle,
        angle_force_kwargs={'k':0.05
            # K is more or less arbitrary, k=4 corresponds to presistence length of 4,
            # k=1.5 is recommended to make polymer realistically flexible; k=8 is very stiff
        },

        nonbonded_force_func=forces.grosberg_repulsive_force,
        nonbonded_force_kwargs={
            'trunc':5.0, # this will let chains cross sometimes
            #'trunc':10.0, # this will resolve chain crossings and will not let chain cross anymore
        },

        except_bonds=False,
    )
)


sim._apply_forces()
sim.local_energy_minimization()


for i in range(19):        
    """
    Here we pass two extra records: a string and an array-like object.
    First becomes an attr, and second becomes an HDF5 dataset
    """
    sim.do_block(1000, save_extras={"eggs": "I don't eat green eggs and ham!!!", "spam":[1,2,3]})

"""
Here we are not forgetting to dump the last set of blocks that the reporter has. 
We have to do it at the end of every simulation. 

I tried adding it to the destructor to make it automatic,
but some weird interactions with garbage collection made it not very useable. 
"""
reporter.dump_data()




INFO:root:adding force grosberg_polymer 0
INFO:root:adding force grosberg_angle 1
INFO:root:adding force grosberg_repulsive 2
INFO:root:Particles loaded. Potential energy is 21.892763
INFO:root:Performing local energy minimization
INFO:root:before minimization eK=1.556626627202928, eP=21.892762645239333, time=0.0 ps
INFO:root:after minimization eK=1.556626627202928, eP=10.835400528930272, time=0.0 ps
INFO:root:block    0 pos[1]=[13.6 13.3 13.0] dr=1.74 t=49.5ps kin=1.57 pot=11.44 Rg=3.324 dt=49.2fs dx=13.77pm 
INFO:root:block    1 pos[1]=[12.3 13.3 12.8] dr=2.58 t=100.3ps kin=1.55 pot=11.16 Rg=4.050 dt=49.7fs dx=13.83pm 
INFO:root:block    2 pos[1]=[11.5 14.4 12.2] dr=2.50 t=151.4ps kin=1.48 pot=11.20 Rg=4.694 dt=50.4fs dx=13.70pm 
INFO:root:block    3 pos[1]=[11.2 16.9 11.3] dr=2.17 t=201.9ps kin=1.45 pot=11.13 Rg=4.633 dt=52.1fs dx=13.99pm 
INFO:root:block    4 pos[1]=[11.9 14.8 12.9] dr=1.98 t=252.2ps kin=1.36 pot=11.24 Rg=4.587 dt=50.9fs dx=13.25pm 
INFO:root:block    5 pos[1]=[11.